# ruGPT

In [ ]:
import numpy as np
import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
def load_tokenizer_and_model(model_name_or_path):
  return GPT2Tokenizer.from_pretrained(model_name_or_path), GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()


def generate(
    model, tok, text,
    do_sample=True, max_length=50, repetition_penalty=5.0,
    top_k=5, top_p=0.95, temperature=1,
    num_beams=None,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3small_based_on_gpt2")
generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)
print(generated)

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt2large")
generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)
generated[0]

# Pipeline
https://github.com/huggingface/notebooks/blob/main/transformers_doc/en/quicktour.ipynb


`pipeline(task=“text-generation”)`

In [ ]:
import torch 

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model="ai-forever/rugpt3small_based_on_gpt2", device=device)

In [ ]:
pipe('Когда родился Александр Сергеевич Пушкин', max_length=250, pad_token_id=50256, num_return_sequences=1)

In [ ]:
from transformers import AutoConfig

my_config = AutoConfig.from_pretrained("distilbert-base-uncased", n_heads=12)

In [ ]:
my_config

# Langchain

https://github.com/curiousily/Get-Things-Done-with-Prompt-Engineering-and-LangChain/blob/master/11.chatbot-with-local-llm-falcon-7b.ipynb

In [1]:
!pip install -qqq langchain --progress-bar off

In [2]:
import re
import warnings
from typing import List

import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)

warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
MODEL_NAME = "tiiuae/falcon-7b-instruct"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, trust_remote_code=True, load_in_8bit=True, device_map="auto"
)
model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"Model device: {model.device}")

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.




A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model device: cuda:0


In [4]:
generation_config = model.generation_config
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "bos_token_id": 11,
  "eos_token_id": 11,
  "max_new_tokens": 256,
  "pad_token_id": 11,
  "repetition_penalty": 1.7,
  "temperature": 0,
  "use_cache": false
}

In [5]:
prompt = """
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context.

Current conversation:

Human: Who is Dwight K Schrute?
AI:
""".strip()
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)

In [6]:
%%time

with torch.inference_mode():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
    )

CPU times: user 7.02 s, sys: 845 ms, total: 7.87 s
Wall time: 16 s


In [7]:
outputs

tensor([[  487,  1863,   304,   241,  6350,  5501,  1192,   241,  2079,   273,
           267,  8317,    25,   390,  8317,   304,  1719,   998,   273,  2665,
          4587,   275,  2005,  2861,   427,   701,  4436,    25,   193,   193,
          9797,  5501,    37,   193,   193, 23431,    37,  5856,   304, 55351,
           531,  2519,    93,  1220,    42,   193, 17362,    37, 55351,   531,
          2519,    93,  1220,   304,   241, 27184,  2655,   272,   248,  2281,
          8203,  2559,   204,    13,   487,  4966,  3566,   707,   304, 29681,
           431, 10601, 13789,    89, 11919,   273,  5741,   271,   314,  3766,
         12579,    23,   480, 21211, 15644,   273,  1764,  3592, 48263, 12674,
          2489,    25,   193,  7932,   204,    11]], device='cuda:0')

In [8]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context.

Current conversation:

Human: Who is Dwight K Schrute?
AI: Dwight K Schrute is a fictional character in the American television series "The Office". He is portrayed by actor Rainn Wilson and appears to be highly intelligent, but socially awkward and often mispronounces words.
User 


In [9]:
class StopGenerationCriteria(StoppingCriteria):
    def __init__(
        self, tokens: List[List[str]], tokenizer: AutoTokenizer, device: torch.device
    ):
        stop_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]
        self.stop_token_ids = [
            torch.tensor(x, dtype=torch.long, device=device) for x in stop_token_ids
        ]

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_ids in self.stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                return True
        return False
     

In [10]:
stop_tokens = [["Human", ":"], ["AI", ":"]]
stopping_criteria = StoppingCriteriaList(
    [StopGenerationCriteria(stop_tokens, tokenizer, model.device)]
)

In [11]:
generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    stopping_criteria=stopping_criteria,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)

In [14]:
%%time
res = llm(prompt)
print(res)

 Dwight K Schrute is a fictional character in the American television series "The Office". He is portrayed by actor Rainn Wilson and appears to be highly intelligent, but socially awkward and often mispronounces words.
User 
CPU times: user 6.62 s, sys: 37.2 ms, total: 6.66 s
Wall time: 6.66 s


In [15]:
chain = ConversationChain(llm=llm)
print(chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [16]:
template = """
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
{history}
Human: {input}
AI:""".strip()

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

print(template)

The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
{history}
Human: {input}
AI:


In [17]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)

chain = ConversationChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

In [18]:
text = "Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce."
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:

Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI:

> Finished chain.
 V8 Family Cars
User 


In [19]:


class CleanupOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        user_pattern = r"\nUser"
        text = re.sub(user_pattern, "", text)
        human_pattern = r"\nHuman:"
        text = re.sub(human_pattern, "", text)
        ai_pattern = r"\nAI:"
        return re.sub(ai_pattern, "", text).strip()

    @property
    def _type(self) -> str:
        return "output_parser"

In [20]:

memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    output_parser=CleanupOutputParser(),
    verbose=True,
)
     

In [21]:
text = "Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce."
res = chain(text)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:

Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI:

> Finished chain.


In [22]:
print(res["response"])

V8 Family Cars


In [23]:

text = "Think of a slogan for the company"
res = chain(text)
print(res["response"])
     



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: V8 Family Cars
Human: Think of a slogan for the company
AI:

> Finished chain.
Built With Power


In [24]:
text = "Choose a domain name for the company"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: V8 Family Cars
Human: Think of a slogan for the company
AI: Built With Power
Human: Choose a domain name for the company
AI:

> Finished chain.
V8FamilyCars.com
